<a href="https://colab.research.google.com/github/Flychuban/Space-Soup/blob/main/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
!pip install git+https://github.com/suno-ai/bark.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import requests
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from bs4 import BeautifulSoup
import re

In [4]:
# Load summarization model from HuggingFace
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Get text from web page

In [5]:
from datetime import datetime
def search_urls_space_news():
  day = int(datetime.now().day)
  month= int(datetime.now().month)
  year = int(datetime.now().year)

  URL = f"https://www.google.com/search?q=space.com+latest+news&tbm=nws"
  print(URL)
  r = requests.get(URL)
  soup = BeautifulSoup(r.text, 'html.parser')
  atags = soup.find_all(['a'])
  hrefs = [link['href'] for link in atags]
  return hrefs


In [6]:
hrefs = search_urls_space_news()

https://www.google.com/search?q=space.com+latest+news&tbm=nws


In [7]:
print(hrefs)

['/?sa=X&ved=0ahUKEwjerLTh09CBAxUImmoFHWsmBsIQOwgC', '/search?q=space.com+latest+news&tbm=nws&sca_esv=569475139&ie=UTF-8&gbv=1&sei=4jIXZd6zEIi0qtsP68yYkAw', '/search?q=space.com+latest+news&sca_esv=569475139&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwjerLTh09CBAxUImmoFHWsmBsIQ_AUIBSgA', '/search?q=space.com+latest+news&sca_esv=569475139&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwjerLTh09CBAxUImmoFHWsmBsIQ_AUIBygC', '/search?q=space.com+latest+news&sca_esv=569475139&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwjerLTh09CBAxUImmoFHWsmBsIQ_AUICCgD', 'https://maps.google.com/maps?q=space.com+latest+news&um=1&ie=UTF-8&sa=X&ved=0ahUKEwjerLTh09CBAxUImmoFHWsmBsIQ_AUICSgE', '/search?q=space.com+latest+news&sca_esv=569475139&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwjerLTh09CBAxUImmoFHWsmBsIQ_AUICigF', '/search?q=space.com+latest+news&sca_esv=569475139&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwjerLTh09CBAxUImmoFHWsmBsIQ_AUICygG', '/advanced_search', '/search?q=space.com+latest+news&sca_esv=56

In [8]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [9]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [10]:
cleaned_urls = strip_unwanted_urls(hrefs, exclude_list)
cleaned_urls

['https://www.space.com/news/live/international-space-station-live-updates',
 'https://www.tweaktown.com/news/93488/scientists-gasped-after-they-opened-the-lid-of-nasas-asteroid-sample-capsule/index.html',
 'https://www.space.com/iran-launches-noor-3-imaging-satellite-september-2023',
 'https://www.space.com/spacex-starlink-launch-group-6-19',
 'https://www.space.com/osiris-rex-asteroid-sample-caniser-lid-lifted-photos',
 'https://www.space.com/james-webb-space-telescope-evolved-galaxy-early-universe',
 'https://www.space.com/china-yaogan-spy-satellite-launch-september-2023',
 'https://www.space.com/harvest-full-moon-will-illuminate-the-sky-on-september-29',
 'https://www.google.com/search?q%3Dspace.com%2Blatest%2Bnews%26tbm%3Dnws%26pccc%3D1',
 'https://www.space.com/historic-donut-black-hole-is-spinning-new-study-finds',
 'https://www.space.com/spacex-launch-psyche-asteroid-mission-one-week']

In [11]:
def get_text_from_urls(urls):
  articles = []
  titles = []
  for url in urls:
    headers = {'User-Agent': 'Mozilla/5.0'}
    cookies = {"CONSENT": "YES+cb.20210720-07-p0.en+FX+410"}
    r = requests.get(url, headers=headers, cookies=cookies)
    if r.status_code != 200:
      continue

    soup = BeautifulSoup(r.text, 'html.parser')
    paragraphs = soup.find_all(['p'])
    text = [paragraph.text for paragraph in paragraphs]
    # words = ' '.join(text)
    article = ' '.join(text)
    article = article.rstrip()

    # Minimal number of symbols in article, prevent returning cookies or terms error
    if len(article) < 200:
      continue

    articles.append(article)
    url_h1_tags = soup.find_all(['h1'])
    url_titles = [title.text.strip() for title in url_h1_tags]
    titles.append(url_titles)

  return titles, articles

In [12]:
titles, articles = get_text_from_urls(cleaned_urls)

In [27]:
articles[1]

'NASA made history this week when it safely landed asteroid samples for the very first time back on Earth, paving the way for researchers to develop a deeper understanding of these space rocks and the solar system\'s evolution. NASA\'s OSIRIS-REx spacecraft delivered the capsule containing samples from the asteroid Bennu to the Utah desert at 10:52 a.m. EDT on September 24, 2023. Notably, Bennu is located approximately 4 billion miles away from Earth, and is the first asteroid sample NASA has ever collected, marking a milestone for the space agency. The container containing the samples was immediately transported to NASA\'s Johnson Space Center in Houston, Texas, and has now been cracked open. According to a new blog post on NASA\'s website and posts on the space agency\'s social media channels, the lid was taken off the canister, and inside was "dark powder and sand-sized particles on the inside of the lid and base," per the NASA Astromaterials X account. The container is expected to 

Split text from the article into chunks

In [14]:
sentences = []
for i, article in enumerate(articles):
  article = article.replace('.', '.<eos>')
  article = article.replace('!', '!<eos>')
  article = article.replace('?', '?<eos>')
  sentences.append(article.split('<eos>'))

['When you purchase through links on our site, we may earn an affiliate commission.',
 ' Here’s how it works.',
 " The liftoff, the eighth for SpaceX's Falcon Heavy rocket, is scheduled for Oct.",
 ' 12.',
 ' Update for Sept.',
 ' 29: NASA has announced that Psyche will now launch on Oct.',
 ' 12.',
 ' This piece has been updated to reflect the new launch date.',
 " We're just two weeks away from the first-ever interplanetary mission for SpaceX's Falcon Heavy rocket.",
 "\xa0 The Falcon Heavy is scheduled to launch NASA's Psyche spacecraft from the agency's Kennedy Space Center in Florida on Oct.",
 ' 12.',
 ' It will be the eighth mission overall for the powerful SpaceX launcher, which debuted in February 2018, and its first for NASA.',
 " Related: NASA's Psyche asteroid probe on track for October launch after 1-year delay The launch window extends through Oct.",
 ' 25, and NASA and SpaceX may need the margin given the looming possibility of a government shutdown on Oct.',
 ' 1.',
 '\

In [15]:
max_tokens_chunk = 500
chunks = []

for i, article_sentences in enumerate(sentences):
  chunks.append([])
  current_chunk = 0
  for sentence in article_sentences:
    if len(chunks[i]) == current_chunk + 1:
      if len(chunks[i][current_chunk]) + len(sentence.split(' ')) <= max_tokens_chunk:
        chunks[i][current_chunk].extend(sentence.split(' '))
      else:
        current_chunk += 1
        chunks[i].append(sentence.split(' '))
    else:
      print(current_chunk)
      chunks[i].append(sentence.split(' '))

0
0
0
0
0
0
0
0
0
0


In [16]:
joined_chunks = []

for i, article_chunks in enumerate(chunks):
    article_text = []

    for chunk_id in range(len(article_chunks)):
        article_text.append(' '.join(article_chunks[chunk_id]))

    joined_chunks.append(article_text)

In [17]:
joined_chunks[0][1]

'\xa0 The astronauts for Expeditions 69/70: NASA astronaut Jasmin Moghbeli, the second Iranian-American in space; European Space Agency (ESA) astronaut Andreas Mogensen; Japan Aerospace Exploration Agency (JAXA) astronaut Satoshi Furukawa; and Konstantin Borisov of Russia\'s space agency,\xa0Roscosmos.  Read more: How to watch SpaceX launch Crew-7 astronauts for NASA early on Aug.  25 with free livestream  An orbiting astronaut found out about Chandrayaan-3\'s successful landing on the\xa0moon\xa0today (Aug.  23) on social media.  It was "all over the news" during the lunch break of Sultan Al Neyadi, an\xa0International Space Station\xa0astronaut conducting the first long-duration orbital mission for the United Arab Emirates (UAE).  "It was really big," the\xa0SpaceX\xa0Crew-6 astronaut told Space. com from orbit during a press conference today.  "I saw multiple clips of mission control in India . . .  it was really great watching that achievement, and, hopefully, as I\'ve mentioned, m

Summarize all chunks

In [18]:
def summarize_chunks(chunks):
  summarized_texts = []
  for article_chunks in chunks:
    print(len(article_chunks))
    article_text = []
    result = summarizer(article_chunks, max_length=120, min_length=30, do_sample=False)
    article_text = ' '.join([summ['summary_text'] for summ in result])
    summarized_texts.append(article_text)
  return summarized_texts

In [19]:
summarized_texts = summarize_chunks(joined_chunks)

35
1
2
2
2
2
2
3
2
2


In [26]:
summarized_texts[-1]

" The launch window for NASA's Psyche asteroid probe has been extended to Oct. 25 . The mission was originally supposed to launch last year, but it was pushed back after problems were discovered with the spacecraft's flight software . It will be the eighth mission overall for the powerful Falcon Heavy rocket, which debuted in February 2018 .  Michael Wall is a Senior Space Writer with Space.com and joined the team in 2010 . He primarily covers exoplanets, spaceflight and military space, but has been known to dabble in the space art beat . Before becoming a science writer, he worked as a herpetologist and wildlife biologist ."

In [21]:
all_summarized_text = ""
for summarized_text in summarized_texts:
  all_summarized_text += summarized_text

In [22]:
all_summarized_text

' Astronaut Frank Rubio and two Russian cosmonauts are scheduled to return to Earth Wednesday (Sept. 27) after a year in space . SpaceX\'s Crew-7 astronaut launch to the International Space Station has been pushed back to Saturday (Aug. 26) You can watch all these events live here at Space. com, courtesy of NASA.com .  Astronaut found out about Chandrayaan-3\'s successful landing on the moon today (Aug. 23) on social media . India\'s historic historic moon landing celebrated by ISS astronauts . The crew for Expeditions 69/70 will head to the International Space Station no earlier than Aug. 25 . On board is Jasmin Moghbeli, the second Iranian-American to reach space on Crew-7 .  A SpaceX Dragon cargo spacecraft is scheduled to undock from the International Space Station today (June 29) All members of Expedition 69 on the ISS were safe during a brief power outage at NASA Tuesday (July 25) NASA\'s Mission Control in Houston briefly lost power and is investigating the cause of the 90-minut

Create a news article with all of the summarized texts

In [ ]:
# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

In [ ]:
# summarized_texts

In [ ]:
# def make_news_summary(all_chunks):
#   summary_space_news = ""
#   for chunk in all_chunks:
#     inputs = tokenizer(f"Make this text like it will be used in space news report: {chunk}", return_tensors='pt', truncation=True, padding=True)
#     outputs = model.generate(**inputs, max_length=512, min_length=len(inputs['input_ids'][0]), temperature=0.25)

#     # Convert the generated output tensor to a string
#     summary_text = tokenizer.decode(outputs[0], skip_special_tokens=True, max_length=512)
#     print(outputs)
#     summary_space_news += summary_text + '\n'
#   return summary_space_news

In [ ]:
# summary_space_news = make_news_summary(summarized_texts)

In [ ]:
#summary_space_news

NLP for news reading

In [23]:
!pip install gTTS

In [24]:
from gtts import gTTS
tts = gTTS(text=all_summarized_text, lang='en')
filename='voice.mp3'
tts.save(filename)